# **Layerwise Relevance Propagation**

We use iNNvestigate to test various interpretability methods, including a whole range of LRP methods.

In [ ]:
!pip install innvestigate

Reinstall Keras 
(as installing iNNvestigate downgrades Keras version)

In [ ]:
!pip install keras==2.3.1

Import necessary packages

In [ ]:
import innvestigate
import innvestigate.utils
from keras.models import load_model
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm_notebook
import os
from scipy import ndimage

List of methods supported by iNNvestigate

In [ ]:
methods = \
['lrp.z_plus',                 
'deep_taylor.bounded',         
'lrp',                         
'lrp.z_IB',                    
'deep_taylor',                 
'lrp.z',                       
'lrp.alpha_1_beta_0_IB',       
'lrp.sequential_preset_a_flat',
'smoothgrad',                   
'input_t_gradient',            
'lrp.alpha_2_beta_1_IB',        
'lrp.alpha_2_beta_1',           
'lrp.epsilon_IB', 
'gradient', 
'pattern.attribution', 
'lrp.sequential_preset_a', 
'deconvnet', 
'integrated_gradients', 
'lrp.epsilon', 
'guided_backprop', 
'lrp.z_plus_fast', 
'lrp.w_square', 
'random', 
'pattern.net', 
'lrp.sequential_preset_b_flat', 
'deep_lift.wrapper', 
'lrp.alpha_1_beta_0', 
'input', 
'lrp.sequential_preset_b', 
'lrp.flat', 
'lrp.alpha_beta', 
'gradient.baseline']

len(methods)

Add your image path in the cell below

In [ ]:
img_path = 'YOUR IMAGE PATH'

Preprocess input

In [ ]:
x = plt.imread(img_path)
plt.imshow(x)
x = x/255
x = np.expand_dims(x,0)
x.shape

Load your Keras model below

In [ ]:
# Add model path below
model = load_model('YOUR MODEL PATH')

Get model predictions

In [ ]:
model.predict(x)

In [ ]:
# Rename your model layers as shown below or however else you wish to prevent occurence of duplicate layer names
for i, layer in enumerate(model.layers):
    layer.name = 'layer_' + str(i)

Choose your method

In [ ]:
my_method = 'CHOOSE YOUR METHOD FROM PREVIOUS LIST' # eg: 'lrp.z_plus', 'deep_taylor.bounded'
print('Chosen method: ',my_method)

Create Analyzer

In [ ]:
# Create Analyzer object
analyzer = innvestigate.create_analyzer(my_method, model)
# Analyze result
a = analyzer.analyze(x)

a = a.sum(axis=3)
a /= np.max(np.abs(a))

Visualize Results on a specific method

In [ ]:
plt.figure(facecolor='white')
plt.imshow(plt.imread(img_path))
plt.imshow(a[0], cmap="seismic", clim=(-1, 1), alpha =0.5)
plt.title('Method: ',my_method)

Visualize multiple methods side by side

In [ ]:
plt.figure(figsize=(80,80), facecolor='white')

for i in tqdm_notebook(range(16)):
    try:
        analyzer = innvestigate.create_analyzer(methods[i], model)
        a = analyzer.analyze(x)
        a = a.sum(axis=3)
        a /= np.max(np.abs(a))
        # Plot
        plt.subplot(4,8,i+1)
        plt.axis('off')
        plt.imshow(plt.imread(img_path))
        plt.imshow(a[0], cmap="seismic", clim=(-1, 1), alpha =0.5)
        plt.title(methods[i])
    except:
        pass
    fig.tight_layout(h_pad=0, w_pad=0)
    plt.subplots_adjust(wspace=0, hspace=0)

# **Testing Sensitivity to Gaussian Noise and Affine Transformations**

Function outlining the perturbed input

In [ ]:
def noise(img,value):
    if(value==0):
        t = img
        t = t[:,:,::-1]
    if(value==1):
        t = cv2.GaussianBlur(img,(5,5),cv2.BORDER_DEFAULT)
        t = t[:,:,::-1]
    if(value==2):
        t = ndimage.rotate(img,90,reshape=False)
        t = t[:,:,::-1]
    if(value==3):
        t = ndimage.rotate(img,180,reshape=False)
        t = t[:,:,::-1]
    if(value==4):
        t = ndimage.rotate(img,270,reshape=False)
        t = t[:,:,::-1]
    return t

Function operating LRP on perturbed input

In [ ]:
def inn_noise(img,value):
    x = noise(img,value)
    x = (x/255).astype(np.float32)
    x = np.expand_dims(x, axis=0)
    a = analyzer.analyze(x)
    a = a.sum(axis=3)
    a /= np.max(np.abs(a))
    return a[0]

Visualize results

In [ ]:
img = cv2.imread(img_path)
# Add your method below
method = 'YOUR METHOD GOES HERE'
# Create analyzer for above method
analyzer = innvestigate.create_analyzer(method, model)

fig = plt.figure(figsize=(50, 50), facecolor='white')
for i in range(1,11):
    plt.subplot(2,5,i)
    if i<6:
        plt.imshow(noise(img, i-1))
    else:
        plt.imshow(noise(img,i-6))
        plt.imshow(inn_noise(img,i-6), cmap="seismic", clim=(-1, 1), alpha =0.5)
    
    plt.axis('off')
    fig.tight_layout(h_pad=0, w_pad=0)
    plt.subplots_adjust(top = 0.99, bottom=0.01, wspace=0, hspace=0)
    fig.suptitle("Method: " + method, fontsize=100)